In [1]:

# import the necessary packages
from TheLazyCoder import social_distancing_config as config
from TheLazyCoder.detection import detect_people
from scipy.spatial import distance as dist
import numpy as np
import argparse
import imutils
import cv2
import os

In [2]:
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--input", type=str, default="",
	help="path to (optional) input video file")
ap.add_argument("-o", "--output", type=str, default="",
	help="path to (optional) output video file")
ap.add_argument("-d", "--display", type=int, default=1,
	help="whether or not output frame should be displayed")
args = vars(ap.parse_args())

usage: ipykernel_launcher.py [-h] [-i INPUT] [-o OUTPUT] [-d DISPLAY]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\anand\AppData\Roaming\jupyter\runtime\kernel-9a5bcdd5-5eca-41bf-a55e-88f495fe5aa1.json


SystemExit: 2

C:\Users\anand\anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3369: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
# load the COCO class labels our YOLO model was trained on
labelsPath = os.path.sep.join([config.MODEL_PATH, "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")

# derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join([config.MODEL_PATH, "yolov3.weights"])
configPath = os.path.sep.join([config.MODEL_PATH, "yolov3.cfg"])

In [4]:
# load our YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)


[INFO] loading YOLO from disk...


In [5]:
ln = net.getLayerNames()

In [7]:
len(ln)

254

In [9]:
type(ln)

tuple

In [10]:
ln

('conv_0',
 'bn_0',
 'leaky_1',
 'conv_1',
 'bn_1',
 'leaky_2',
 'conv_2',
 'bn_2',
 'leaky_3',
 'conv_3',
 'bn_3',
 'leaky_4',
 'shortcut_4',
 'conv_5',
 'bn_5',
 'leaky_6',
 'conv_6',
 'bn_6',
 'leaky_7',
 'conv_7',
 'bn_7',
 'leaky_8',
 'shortcut_8',
 'conv_9',
 'bn_9',
 'leaky_10',
 'conv_10',
 'bn_10',
 'leaky_11',
 'shortcut_11',
 'conv_12',
 'bn_12',
 'leaky_13',
 'conv_13',
 'bn_13',
 'leaky_14',
 'conv_14',
 'bn_14',
 'leaky_15',
 'shortcut_15',
 'conv_16',
 'bn_16',
 'leaky_17',
 'conv_17',
 'bn_17',
 'leaky_18',
 'shortcut_18',
 'conv_19',
 'bn_19',
 'leaky_20',
 'conv_20',
 'bn_20',
 'leaky_21',
 'shortcut_21',
 'conv_22',
 'bn_22',
 'leaky_23',
 'conv_23',
 'bn_23',
 'leaky_24',
 'shortcut_24',
 'conv_25',
 'bn_25',
 'leaky_26',
 'conv_26',
 'bn_26',
 'leaky_27',
 'shortcut_27',
 'conv_28',
 'bn_28',
 'leaky_29',
 'conv_29',
 'bn_29',
 'leaky_30',
 'shortcut_30',
 'conv_31',
 'bn_31',
 'leaky_32',
 'conv_32',
 'bn_32',
 'leaky_33',
 'shortcut_33',
 'conv_34',
 'bn_34',
 'l

In [11]:
Ul = net.getUnconnectedOutLayers()

In [12]:
Ul

array([200, 227, 254])

In [13]:
type(Ul)

numpy.ndarray

In [15]:
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]

In [16]:
ln

['yolo_82', 'yolo_94', 'yolo_106']

In [17]:
vs = cv2.VideoCapture("pedestrians.mp4")

In [18]:
writer = None

In [22]:
(grabbed, frame) = vs.read()


In [23]:
frame

array([[[110, 104, 112],
        [112, 106, 114],
        [113, 107, 115],
        ...,
        [210, 222, 221],
        [207, 219, 218],
        [206, 218, 217]],

       [[117, 111, 119],
        [123, 117, 125],
        [126, 120, 128],
        ...,
        [210, 222, 221],
        [206, 218, 217],
        [205, 217, 216]],

       [[135, 129, 137],
        [142, 136, 144],
        [148, 142, 150],
        ...,
        [209, 221, 220],
        [205, 217, 216],
        [204, 216, 215]],

       ...,

       [[ 93,  99, 115],
        [ 94, 100, 116],
        [ 97, 103, 119],
        ...,
        [ 59,  60,  49],
        [ 78,  79,  68],
        [ 97,  98,  87]],

       [[ 93,  99, 115],
        [ 94, 100, 116],
        [ 97, 103, 119],
        ...,
        [ 59,  60,  49],
        [ 78,  79,  68],
        [ 97,  98,  87]],

       [[ 93,  99, 115],
        [ 94, 100, 116],
        [ 97, 103, 119],
        ...,
        [ 59,  60,  49],
        [ 78,  79,  68],
        [ 97,  98,  87]]

In [24]:
frame.shape

(1080, 1920, 3)

In [25]:
frame = imutils.resize(frame, width=700)

In [26]:
frame.shape

(393, 700, 3)

In [30]:
(H, W) = frame.shape[:2]
results = []

blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),swapRB=True, crop=False)

In [32]:
blob.shape

(1, 3, 416, 416)

In [33]:
net.setInput(blob)
layerOutputs = net.forward(ln)


In [36]:
len(layerOutputs)

3

In [39]:
print(len(layerOutputs[0]))
print(len(layerOutputs[1]))
print(len(layerOutputs[2]))


507
2028
8112


In [43]:
print(len(layerOutputs[0][5]))
print(len(layerOutputs[0][6]))
print(len(layerOutputs[0][7]))
print(len(layerOutputs[0][500]))

85
85
85
85


In [45]:
results = detect_people(frame, net, ln,personIdx=LABELS.index("person"))

In [47]:
results

[(0.9976577162742615, (563, 253, 603, 357), (583, 305)),
 (0.9950392842292786, (2, 205, 37, 287), (20, 246)),
 (0.9908084869384766, (467, -2, 487, 40), (477, 19)),
 (0.9903967380523682, (171, 97, 197, 171), (184, 134)),
 (0.9845964312553406, (87, 113, 116, 180), (102, 147)),
 (0.968180239200592, (343, 337, 393, 391), (368, 364)),
 (0.9399741291999817, (479, 100, 513, 159), (496, 130)),
 (0.9253908395767212, (493, 0, 515, 45), (504, 23)),
 (0.9173200130462646, (210, 65, 231, 117), (221, 91)),
 (0.8434418439865112, (425, -2, 438, 19), (432, 8)),
 (0.8394718170166016, (484, -1, 503, 40), (494, 19)),
 (0.3375735282897949, (666, 89, 683, 125), (675, 107)),
 (0.3147476613521576, (352, 3, 367, 48), (360, 26))]

In [61]:
violate = set()

if len(results) >= 2:
	centroids = np.array([r[2] for r in results])
	print(centroids)
	D = dist.cdist(centroids, centroids, metric="euclidean")
	print(D)
    
	for i in range(0, D.shape[0]):
		for j in range(i + 1, D.shape[1]):

			if D[i, j] < config.MIN_DISTANCE:
				# update our violation set with the indexes of
				# the centroid pairs
				violate.add(i)
				violate.add(j)

[[583 305]
 [ 20 246]
 [477  19]
 [184 134]
 [102 147]
 [368 364]
 [496 130]
 [504  23]
 [221  91]
 [432   8]
 [494  19]
 [675 107]
 [360  26]]
[[  0.         566.08303278 305.01147519 434.09906703 506.28549258
  222.94842453 195.43285292 292.85662021 420.52348329 333.18163215
  299.52796197 218.33002542 357.16942758]
 [566.08303278   0.         510.27247623 198.5950654  128.54960132
  367.46156262 489.93060733 532.90243009 253.82277282 475.80248003
  525.55209066 669.58643953 404.96913463]
 [305.01147519 510.27247623   0.         314.7602262  396.24361194
  361.80934206 112.61438629  27.29468813 265.93232222  46.32493929
   17.         216.67487164 117.20921465]
 [434.09906703 198.5950654  314.7602262    0.          83.02409289
  294.54371492 312.02563997 338.70488629  56.72741841 278.17260828
  330.64331235 491.74180217 206.49455199]
 [506.28549258 128.54960132 396.24361194  83.02409289   0.
  343.28559539 394.36658073 420.68990955 131.5180596  358.07960009
  412.368767   574.3944637

In [54]:
centroids.shape

(13, 2)

(13, 13)

In [63]:
violate

{2, 7, 9, 10}

In [64]:
# loop over the results
for (i, (prob, bbox, centroid)) in enumerate(results):
    # extract the bounding box and centroid coordinates, then
    # initialize the color of the annotation
    (startX, startY, endX, endY) = bbox
    (cX, cY) = centroid
    color = (0, 255, 0)

    # if the index pair exists within the violation set, then
    # update the color
    if i in violate:
        color = (0, 0, 255)

    # draw (1) a bounding box around the person and (2) the
    # centroid coordinates of the person,
    cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
    cv2.circle(frame, (cX, cY), 5, color, 1)

In [65]:
text = "Social Distancing Violations: {}".format(len(violate))
cv2.putText(frame, text, (10, frame.shape[0] - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)

array([[[123, 117, 125],
        [133, 127, 135],
        [147, 141, 149],
        ...,
        [224, 232, 231],
        [216, 227, 226],
        [207, 219, 218]],

       [[165, 159, 167],
        [157, 151, 159],
        [144, 138, 146],
        ...,
        [228, 236, 235],
        [216, 227, 226],
        [203, 215, 214]],

       [[148, 143, 151],
        [146, 141, 149],
        [149, 144, 152],
        ...,
        [231, 240, 239],
        [216, 227, 226],
        [200, 212, 211]],

       ...,

       [[ 94, 101, 117],
        [ 98, 104, 120],
        [ 95, 102, 118],
        ...,
        [ 55,  54,  48],
        [ 54,  54,  44],
        [ 76,  77,  66]],

       [[ 94, 100, 116],
        [ 98, 104, 120],
        [ 95, 101, 117],
        ...,
        [ 54,  53,  46],
        [ 54,  55,  45],
        [ 79,  80,  69]],

       [[ 94, 100, 116],
        [ 98, 104, 120],
        [ 95, 101, 117],
        ...,
        [ 52,  51,  44],
        [ 54,  54,  44],
        [ 80,  81,  70]]

In [67]:
cv2.imshow("Frame", frame)
key = cv2.waitKey(1) & 0xFF


